In [ ]:
import random
import covalent as ct
import pennylane as qml
import os

import dotenv
dotenv.load_dotenv()
IBQM_token = os.getenv('IBMQ_token')

# Define a Qiskit Runtime executor.
qiskit_sampler = ct.executor.QiskitExecutor(
        device="sampler",
        backend="ibmq_qasm_simulator",
        single_job=False,
)
qiskit_executor = ct.executor.QiskitExecutor(
    device="sampler",
    backend="ibmq_qasm_simulator",
    ibmqx_token=IBQM_token,  # required if not in config file
    single_job=False,
)


# Create a QElectron that executes on Qiskit Runtime.
@ct.qelectron(executors=qiskit_executor)
@qml.qnode(qml.device("default.qubit", wires=2, shots=1024))
def circuit(x):
    qml.RX(x, wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliY(0))


# Create Electrons for a simple workflow.
@ct.electron
def generate_rand_input():
    return random.uniform(0, 3.14159)

@ct.electron
def run_qiskit_experiment(x):
    # Workflow task that uses the `circuit` QElectron.
    return circuit(x)

@ct.lattice
def workflow():
    results = []
    
    # Six independent experiments.
    for _ in range(6):
        x = generate_rand_input()
        results.append(run_qiskit_experiment(x))

    return results

# Dispatch workflow.
dispatch_id = ct.dispatch(workflow)()
print(dispatch_id)
results = ct.get_result(dispatch_id, wait=True).result

In [3]:
results

[tensor(-0.63671875, requires_grad=True),
 tensor(-0.82421875, requires_grad=True),
 tensor(-0.65625, requires_grad=True),
 tensor(-0.97265625, requires_grad=True),
 tensor(-0.97460938, requires_grad=True),
 tensor(-1., requires_grad=True)]